In [1]:
import semiolog as slg

semiotic = slg.Cenematic("en_bnc_test")

semiotic.corpus.build(
    save = True,
    )

semiotic.vocab.build(
    save = True,
    parallel = True,
    )

semiotic.paradigmatic.build(load_tokenized=True, save_tokenized=True, n_sents=10000)

SLG: Checking config correctness...
SLG: Config correct!
SLG Warning: models/en_bnc_test/corpus/train.txt does not exist.
Corpus will not be loaded from file.

SLG Warning: models/en_bnc_test/vocabulary/merges.txt does not exist.
Vocabulary will not be loaded from file.



Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 326.08it/s]


Dataset text downloaded and prepared to /Users/Gianni/.cache/huggingface/datasets/text/original-b00db825dd6cd38c/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 173.79it/s]


SLG: Dataset loaded from the following files: dev.txt.



...

100%|██████████| 18/18 [00:00<00:00, 20.40ba/s]


SLG: Filtering rows of length <2


100%|██████████| 1/1 [00:00<00:00, 36.40ba/s]


SLG: Building train set
SLG: Building validation set
SLG: Starting training...

Epoch 1/2
1238/1238 [==============================] - 2029s 2s/step - loss: 6.6066 - val_loss: 6.0820
Epoch 2/2
1238/1238 [==============================] - 1939s 2s/step - loss: 5.9536 - val_loss: 5.6393
SLG: Training finished

SLG: Model saved.
SLG: Training history saved.


'SLG: Model built!'

In [2]:
from random import randint
n = randint(0,semiotic.corpus.train.num_rows)
# n = 51900
sent = semiotic.corpus.train[n]["text"]

sent_seq = semiotic(sent)
print(n)
sent_seq.tree.plot()

228102


interactive(children=(Output(),), _dom_classes=('widget-interact',))

<function semiolog.syntagmatic.tree.Tree.plot.<locals>.inter()>

In [3]:
sent_seq.parad_chain.df()

,the_0,long_1,est_2,isthe_3,second_4,d_5,ors_6,almost_7,sp_8,ine_9,...,upto_11,two_12,arm_13,se_14,g_15,ments_16,in_17,l_18,eng_19,th_20
0,‘,e,s,s,s,s,s,s,s,s,...,s,s,s,s,s,s,s,s,s,s
1,the,h,ing,ing,ing,ing,ing,ing,ing,ing,...,ing,ing,ing,ing,ing,ing,ing,ing,ing,ing
2,but,but,to,the,to,to,to,to,to,to,...,to,to,to,to,to,to,to,to,to,to
3,in,f,the,to,the,the,the,the,the,the,...,and,the,the,the,and,the,the,the,the,and
4,and,edbythe,it,it,and,and,and,and,and,and,...,the,and,and,and,the,and,and,and,and,the
5,at,ence,and,and,it,it,it,it,it,it,...,it,it,it,it,it,it,it,it,it,it
6,what,ice,in,in,in,in,in,in,in,in,...,in,in,in,in,in,in,in,in,in,in
7,he,that,t,t,t,ed,ed,ed,t,t,...,t,t,t,ed,t,t,t,ed,ed,t
8,this,bo,is,ed,ed,t,t,t,ed,is,...,ed,ed,ed,t,is,ed,ed,t,t,is
9,as,ic,ed,is,is,is,is,is,is,ed,...,is,is,is,is,ed,is,is,is,is,ed
